You are given an array of strings arr. A string s is formed by the concatenation of a subsequence of arr that has unique characters.

Return the maximum possible length of s.

A subsequence is an array that can be derived from another array by deleting some or no elements without changing the order of the remaining elements.

 

Example 1:

Input: arr = ["un","iq","ue"]
Output: 4
Explanation: All the valid concatenations are:
- ""
- "un"
- "iq"
- "ue"
- "uniq" ("un" + "iq")
- "ique" ("iq" + "ue")
Maximum length is 4.
Example 2:

Input: arr = ["cha","r","act","ers"]
Output: 6
Explanation: Possible longest valid concatenations are "chaers" ("cha" + "ers") and "acters" ("act" + "ers").
Example 3:

Input: arr = ["abcdefghijklmnopqrstuvwxyz"]
Output: 26
Explanation: The only string in arr has all 26 characters.
 

Constraints:

1 <= arr.length <= 16
1 <= arr[i].length <= 26
arr[i] contains only lowercase English letters.

# approach 1:
- take not take method,
- when ind == n :
-   if the charactors are unique, then chage the maxi.

In [ ]:
from collections import Counter

class Solution:
    def __init__(self):
        self.res = [0]

    def maxLength(self, arr: list[str]) -> int:
        self.maxLength_re(arr, 0, "", len(arr))
        return self.res[0]

    def maxLength_re(self, arr: list[int], ind:int, cur_text: str, n: int):
        if ind == n:
            # if all the charactors are unique.. 
            if cur_text and max(Counter(cur_text).values()) == 1:
                print(cur_text)
                self.res[0] = max(self.res[0], len(cur_text))
            return

        # take the current text.
        temp = cur_text
        cur_text += arr[ind]
        self.maxLength_re(arr, ind + 1, cur_text, n)

        # not take.
        cur_text = temp 
        self.maxLength_re(arr, ind + 1, cur_text, n)




#### Breakdown:

* The recursion explores all **subsets** of `arr`, which is $2^n$.
* For each subset, we concatenate strings and then check if the result has **all unique characters**.
* This uniqueness check using `Counter` takes $O(L)$, where $L$ is the length of the concatenated string (at most 26 because only lowercase letters are allowed).

So,

$$
\text{Time Complexity} = O(2^n \cdot L)
$$

> Worst case $L = 26$, so upper bound: $O(2^n \cdot 26) = O(2^n)$

---

### ✅ **Space Complexity**

#### Call stack:  $O(n)$

#### Memory for string operations:

* Each recursive call creates a new string `cur_text`, which could go up to length 26 → $O(26) = O(1)$

#### Counter dictionary:

* `Counter(cur_text)` holds at most 26 characters → $O(1)$

#### Final result stored:

* Just one integer in a list `self.res` → $O(1)$

So,

$$
\text{Space Complexity} = O(n)
$$



In [10]:
Solution().maxLength(arr = ["un","iq","ue"])

0
1
2
3
3
uniq
2
3
3
un
1
2
3
ique
3
iq
2
3
ue
3


4

## Bit manupulation way

- We can convert each char to a 26 bit int... since only lower case english letters are used this is a better aproach.
- pre-process the list to keep on the ele which are having no-repeating characters.
- then do the take- nontake to build the string and only conside the once, having the non-overlapping bits (which is non-repeating charactors.)


In [ ]:
class Solution:
    def maxLength(self, arr: list[str]) -> int:
        # Preprocess to store only valid strings with unique characters.
        masks = []
        for subseqience in arr:
            mask = 0
            is_valid = True
            for ch in subseqience:
                # convert the charctor to a int only sent on the current bit. 
                # 001 - a
                # 010 - b
                
                bit = 1 << (ord(ch) - ord('a'))

                if mask & bit:
                    is_valid = False  # duplicate char in string.
                    break
                # To add the charactor to the mask.
                mask |= bit
            if is_valid:
                masks.append((mask, len(subseqience)))
    
        def maxLength_rec(ind, curr_mask):
            # The recurssion to check the values.
            if ind == len(masks):
                # Count the number of 1, which is the charcter in the mask.
                return bin(curr_mask).count('1')
            
            # not-take the current mask and move on.
            max_len_not_take = maxLength_rec(ind + 1, curr_mask)

            # take the current mask.
            max_len_take = 0 
            mask, length = masks[ind]
            if mask & curr_mask == 0:   # no common 1's to add.
                # so they are unique.
                max_len_take = maxLength_rec(ind + 1, curr_mask | mask)
            
            return max(max_len_not_take, max_len_take)
        
        # Call the recurssion call.
        return maxLength_rec(0, 0)
    
# tc:
# - pre-processing arr: O(n * L)   --> n: len(arr), L - avg(lenght of string)
# - recurssion - O(2^n)
# - tc - O(2^n)

# sc:
# - mask arr - O(n)
# - axilory - O(n)
# - sc - O(n)


In [5]:
Solution().maxLength(arr = ["un","iq","ue"])

4


| Step                | Description                                            | Example Input                  | Operation/Result                                                |
| ------------------- | ------------------------------------------------------ | ------------------------------ | --------------------------------------------------------------- |
| **Preprocessing 1** | Convert `"un"` to bitmask                              | `"un"`                         | `mask = 0b1000000000000000000000010` (bits for `'u'` and `'n'`) |
| **Preprocessing 2** | Check duplicate chars in `"un"`                        | `"un"`                         | No duplicates → valid mask stored                               |
| **Preprocessing 3** | Convert `"iq"` to bitmask                              | `"iq"`                         | `mask = 0b100000000010000` (bits for `'i'` and `'q'`)           |
| **Preprocessing 4** | Check duplicates in `"iq"`                             | `"iq"`                         | No duplicates → valid mask stored                               |
| **Preprocessing 5** | Convert `"ue"` to bitmask                              | `"ue"`                         | `mask = 0b100000000000001` (bits for `'u'` and `'e'`)           |
| **Preprocessing 6** | Check duplicates in `"ue"`                             | `"ue"`                         | No duplicates → valid mask stored                               |
| **Recursion 1**     | Start with `curr_mask = 0` (empty)                     |                                | Explore subsets starting at index 0                             |
| **Recursion 2**     | Pick `"un"` → new mask = `0b1000000000000000000000010` |                                | Valid, recurse next index with combined mask                    |
| **Recursion 3**     | Pick `"iq"` after `"un"`?                              | Check `curr_mask & mask("iq")` | `0b0` → no overlap, valid                                       |
| **Recursion 4**     | Pick `"ue"` after `"un"`?                              | Check `curr_mask & mask("ue")` | Overlaps on `'u'` → skip                                        |
| **Recursion 5**     | Skip `"un"`, pick `"iq"`                               |                                | Valid, recurse with mask("iq")                                  |
| **Recursion 6**     | Pick `"ue"` after `"iq"`?                              | Check `curr_mask & mask("ue")` | No overlap, valid                                               |
| **Result**          | Maximum length from valid combinations                 | `"uniq"` or `"ique"`           | Length = 4                                                      |
